In [9]:
import torch
import numpy as np
import os
import sys
sys.path.append("./src/models/")
sys.path.append("./src/")
import utils
from MWU_CNN import MW_Unet

In [65]:
model_save_path  = "./experiments/baseline/baselin.pt"
def backprop(optimizer,model_output,target):
    optimizer.zero_grad()
    loss = nn.MSELoss()
    loss.backward()
    optimizer.step()
def get_PSNR(model_output,target):
    I_hat = model_output.numpy()
    I = target.numpy()
    mse = (np.square(I-I_hat)).mean(axis=None)
    PSNR = 10 * np.log10(1.0/mse)
    return PSNR
def train(args):
    """
    train model
    """
    
    ####################################### Initializing Model #######################################
    
    step = 0.01
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print_every = int(args['--print_every'])
    num_epochs = int(args['--num_epochs'])
    save_every = int(args['--save_every'])
    save_path = model_save_path
    num_batches = int(args['--num_batches'])
    
    model = MW_Unet(num_conv=3,in_ch=1)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=step)
    
    ######################################### Loading Data ##########################################
    train_data_path = args['--train_data_path']
    n  = 1000
    dataset_total = utils.patchesDataset(patches_path=train_data_path,n=n)
    train_split,val_split,test_split = 0.8,0.1,0.1
    train_max_idx = int(train_split*len(dataset_total))
#     print(train_max_idx)
    dataset_train = torch.utils.data.Subset(dataset_total,range(0,train_max_idx))
    val_max_idx = train_max_idx+int(val_split*len(dataset_total))
    dataset_val = torch.utils.data.Subset(dataset_total,range(train_max_idx,val_max_idx))
    test_max_idx = val_max_idx + int(test_split*len(dataset_total))
#     print(test_max_idx)
    dataset_test = torch.utils.data.Subset(dataset_total,range(val_max_idx,test_max_idx))
    
    
    print("length of train set: ",len(dataset_train))
    print("length of val set: ",len(dataset_val))
    print("length of test set: ",len(dataset_test))
    
    train_PSNRs = []
    train_losses = []
    val_accs = []
    
    
    


In [66]:
args = {'--print_every':1,'--num_epochs':1,'--save_every':1,'--num_batches':32,'--train_data_path':'./data/patches_Train/'}
train(args)

  0%|          | 0/10000 [00:00<?, ?it/s]

loading patches from patches directory


100%|██████████| 10000/10000 [02:14<00:00, 74.09it/s]


completed loading patches from directory!
(10000, 240, 240)
shape of target: (10000, 1, 240, 240) shape of noisy: (10000, 1, 240, 240)
length of train set:  8000
length of val set:  1000
length of test set:  1000
